In [32]:
import pyomo.environ as pyo

# Create Pyomo model
model = pyo.ConcreteModel()

# Sets
states = ['A', 'B', 'C']

# Parameters
demand = {'A': 5000, 'B': 7000, 'C': 4000}  # MWh
solar_max_capacity = {'A': 5, 'B': 2, 'C': 8}  # MW
gas_existing_capacity = {'A': 1, 'B': 2, 'C': 1}  # MW
solar_investment_cost = 300  # $/MW
solar_operating_cost = 10  # $/MWh
gas_operating_cost = 20  # $/MWh
gas_investment_cost = 150  # $/MW
emissions_factor_gas = 0.5  # tons CO2/MWh
emissions_cap = 2000  # tons CO2
hours_per_year = 8760  # hours in a year
transmission_cost = 0  # $/MWh

transmission_capacity = {
    ('A', 'B'): 2000,  
    ('B', 'A'): 2000,
    ('B', 'C'): 1500,
    ('C', 'B'): 1500,   
}

# Decision pyo.Variables
model.solar_generation = pyo.Var(states, within=pyo.NonNegativeReals)  # MWh
model.solar_capacity = pyo.Var(states, within=pyo.NonNegativeReals, bounds=(0, max(solar_max_capacity.values())))  # MW
model.gas_capacities = pyo.Var(states, within=pyo.NonNegativeReals)  # MW
model.gas_factors = pyo.Var(states, within=pyo.NonNegativeReals, bounds=(0, 1))  # % (between 0-100%)

# Transmission pyo.variables
model.transmission = pyo.Var(transmission_capacity.keys(), within=pyo.NonNegativeReals)
model.transmission_reverse = pyo.Var(transmission_capacity.keys(), within=pyo.NonNegativeReals)

# Objective Function: Minimize total cost
def total_cost(model):
    return sum(
        solar_investment_cost * model.solar_capacity[s] +
        solar_operating_cost * model.solar_generation[s] +
        gas_operating_cost * model.gas_capacities[s] +
        gas_investment_cost * (model.gas_capacities[s] - gas_existing_capacity[s])
        for s in states
    ) + sum(transmission_cost * model.transmission[t] for t in transmission_capacity)

model.obj = pyo.Objective(rule=total_cost, sense=pyo.minimize)

# Constraints

# Energy balance constraints
def balance_rule(model, s):
    if s == 'A':
        return model.solar_generation[s] + model.gas_capacities[s] * model.gas_factors[s] * hours_per_year \
               + model.transmission[('B', 'A')] - model.transmission[('A', 'B')] >= demand[s]
    elif s == 'B':
        return model.solar_generation[s] + model.gas_capacities[s] * model.gas_factors[s] * hours_per_year \
               + model.transmission[('A', 'B')] + model.transmission[('C', 'B')] \
               - (model.transmission[('B', 'A')] + model.transmission[('B', 'C')]) >= demand[s]
    elif s == 'C':
        return model.solar_generation[s] + model.gas_capacities[s] * model.gas_factors[s] * hours_per_year \
               + model.transmission[('B', 'C')] - model.transmission[('C', 'B')] >= demand[s]

model.balance_constraint = pyo.Constraint(states, rule=balance_rule)

# Solar generation limit
def solar_limit_rule(model, s):
    return model.solar_generation[s] <= 1000 * solar_max_capacity[s]

model.solar_limit = pyo.Constraint(states, rule=solar_limit_rule)

# Gas capacity at least installed
def gas_capacity_rule(model, s):
    return model.gas_capacities[s] >= gas_existing_capacity[s]

model.gas_capacity_constraint = pyo.Constraint(states, rule=gas_capacity_rule)

# Emissions cap
def emissions_rule(model):
    return sum(emissions_factor_gas * model.gas_capacities[s] * model.gas_factors[s] * hours_per_year for s in states) <= emissions_cap

model.emissions_constraint = pyo.Constraint(rule=emissions_rule)

def transmission_rule(model, from_state, to_state):
    return model.transmission[from_state, to_state] + model.transmission_reverse[from_state, to_state] <= transmission_capacity[from_state, to_state]
# Apply the constraint correctly
model.transmission_constraint = pyo.Constraint(transmission_capacity.keys(), rule=transmission_rule)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Requires IPOPT solver installed
solver.solve(model)

# Extract results
results_with_transmission = {
    "Demand (MWh)": {s: demand[s] for s in states},
    "Solar Generation (MWh)": {s: round(model.solar_generation[s].value, 1) for s in states},
    "Gas Capacities (MWh)": {s: round(model.gas_capacities[s].value, 1) for s in states},
    #"Solar Capacity Installed (MW)": {s: model.solar_capacity[s].value for s in states},
    "Gas Factors (%)": {s: round(model.gas_factors[s].value, 2) for s in states},
    "Transmission (MWh)": {t:round(model.transmission[t].value, 1) for t in transmission_capacity},
    "Total Cost ($)": model.obj()
}


In [33]:
# Print results
for key, value in results_with_transmission.items():
    print(f"{key}: {value}")


Demand (MWh): {'A': 5000, 'B': 7000, 'C': 4000}
Solar Generation (MWh): {'A': 4904.7, 'B': 1918.2, 'C': 5177.1}
Gas Capacities (MWh): {'A': 1.0, 'B': 2.0, 'C': 1.0}
Gas Factors (%): {'A': 0.09, 'B': 0.18, 'C': 0.01}
Transmission (MWh): {('A', 'B'): 859.5, ('B', 'A'): 141.7, ('B', 'C'): 77.2, ('C', 'B'): 1338.4}
Total Cost ($): 120079.99798427518


In [ ]:
def print_results(demand, results):
    states = ['A', 'B', 'C']
    results_with_transmission = {
    "Demand (MWh)": {s: demand[s] for s in states},
    "Solar Generation (MWh)": {s: round(results["solarCapacity{state}"]*1000, 1) for s in states},
    "Gas Capacities (MWh)": {s: round(results["gasCapacity{state}"], 1) for s in states},
    #"Solar Capacity Installed (MW)": {s: model.solar_capacity[s].value for s in states},
    "Gas Factors (%)": {s: round(results["gasFactor{state}"], 1) for s in states},
    "Transmission (MWh)": results[-4:],
    "Total Cost ($)": sum(solar_investment_cost * results["solarCapacity{state}"] + 
        solar_operating_cost * results["solarCapacity{state}"] * 1000 + 
        gas_operating_cost * results["gasCapacity{state}"] * 8760 * results["gasFactor{state}"] + 
        gas_investment_cost * (results["gasCapacity{state}"] - gas_existing_capacity[s])
        for s in states)
}

In [ ]:
results = {
    "gasCapacityA": 6,
    "gasCapacityB": 10,
    "gasCapacityC": 10,
    "gasFactorA": 0.1,
    "gasFactorB": 0.3,
    "gasFactorC": 0.3,
    "solarCapacityA": 2,
    "solarCapacityB": 8,
    "solarCapacityC": 9,
    "transmissionAB": 0,
    "transmissionBA": 1000,
    "transmissionBC": 1000,
    "transmissionCB": 0
  },
